# Converting Models to OpenVINO IR

In this lab, you will learn about converting models to an OpenVINO Immediate Representation (IR). This representation will allow OpenVINO to conduct inference optimally on Intel hardware.

### Sections
1.1 **Convert Keras Models (.hdf5):** We will convert a fault segmentation Keras model to a frozen graph.<br/>
1.2 **Convert Tensorflow Frozen Graphs (.pb):** Using our newly converted frozen graph, we will use the model optimizer to convert from frozen graph to IR. <br/>
1.3 **Convert ONNX Models (.onnx):** We will convert a salt identification Pytorch model to its ONNX equivalent and then convert to IR.<br/>

### Imports

Below are some important imports and global variables that we will need to set up before moving forward. Make sure to run `pip install -r requirements.txt` and build/pull the docker image for Open Seismic.

In [1]:
import os
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import warnings

import numpy as np
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras.layers import *
from keras.models import load_model
import tensorflow.keras.backend as K
import shutil, sys

import torch
from pathlib import PurePath
from keras.models import model_from_json, Sequential

Using TensorFlow backend.


## Section 1.1: Convert Keras Models (.hdf5)

OpenVINO supports popular deep learning frameworks, and [here](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) is a list of the frameworks that it supports. For Keras models, we will first convert to a Tensorflow frozen graph representation. In section 1.2, we will convert the Tensorflow frozen graph representation into OpenVINO's Immediate Representation (IR). Below, we will walk through how to convert a Keras model to a frozen graph.

### Steps
1. Using the Keras library, load the model into memory by specifying the path to the Keras model as well as custom objects that you defined in your network.

In [2]:
vggModel = '../vgg.json'
vggWeights = '../vgg.h5'
mlpModel = '../gsbMLP.json'
mlpWeights = '../gsbMLP.h5'

# jsonModelFilePath, weightsFilePath = mlpModel, mlpWeights
jsonModelFilePath, weightsFilePath = vggModel, vggWeights

# load model
jsonModelFile = open(jsonModelFilePath, 'r' )
base_model = jsonModelFile.read()
jsonModelFile.close()
model = model_from_json(base_model)
model.load_weights(weightsFilePath)
model.compile( loss='binary_crossentropy', optimizer='sgd', metrics=[ 'accuracy' ] )

In [3]:
# path_to_keras = str(assets_path.joinpath('fseg-60.hdf5'))
# model = load_model(path_to_keras, custom_objects={'cross_entropy_balanced': cross_entropy_balanced})
assets_path = PurePath('.')
frozen_model_path = str(assets_path)
frozen_model_name = 'vgg_sfd.pb'

2. Get the node names that exist within the Keras model layers

In [4]:
tf.get_logger().setLevel('INFO')
K.set_image_data_format('channels_last')

output_node_names = [node.op.name for node in model.outputs]

3. Instantiate a Keras session and use the graph_util function from Tensorflow's framework module to create a constant graph.

In [5]:
sess = K.get_session()
constant_graph = graph_util.convert_variables_to_constants(sess,
                                                           sess.graph.as_graph_def(),
                                                           output_node_names)

INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.


4. Finally, write the constant graph to disk with name `frozen_model_name` at `frozen_model_path`.

In [6]:
graph_io.write_graph(constant_graph, frozen_model_path, frozen_model_name, as_text=False)

'./vgg_sfd.pb'

In four easy steps, we were able to convert a Keras model into a frozen graph.

## Section 1.2: Convert Tensorflow Frozen Graphs (.pb)

In the previous section, we walked through how to convert a Keras model into its frozen graph equivalent. This was a necessary step because we can now convert our frozen graph into an OpenVINO IR. Below, we will walk through how to convert a frozen graph model to OpenVINO IR using the Open Seismic Docker image.

### Steps
1. Familiarize yourself with the Model Optimizer options. For a detailed explanation, go [here](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_Config_Model_Optimizer.html).

In [7]:
os_version = 'os:opensource_resmnt'
model_optimizer_cmd = f"""
docker run {os_version} /bin/bash executables/mo.sh -h
"""
! {model_optimizer_cmd}

usage: mo.py [options]

optional arguments:
  -h, --help            show this help message and exit
  --framework {tf,caffe,mxnet,kaldi,onnx}
                        Name of the framework used to train the input model.

Framework-agnostic parameters:
  --input_model INPUT_MODEL, -w INPUT_MODEL, -m INPUT_MODEL
                        Tensorflow*: a file with a pre-trained model (binary
                        or text .pb file after freezing). Caffe*: a model
                        proto file with model weights
  --model_name MODEL_NAME, -n MODEL_NAME
                        Model_name parameter passed to the final create_ir
                        transform. This parameter is used to name a network in
                        a generated IR and output .xml/.bin files.
  --output_dir OUTPUT_DIR, -o OUTPUT_DIR
                        Directory that stores the generated IR. By default, it
                        is the directory from where the Model Optimizer is
                        lau

2. Specify the appropriate configurations.

In [8]:
# Model expects a 5-dimensional input
model.input.get_shape().as_list()

[None, 45, 45, 1]

In [9]:
phys_mnt_vol = str(PurePath(os.getcwd()))
docker_mnt_vol = '/mnt_vol'
docker_frozen_model_path = f'{docker_mnt_vol}/{frozen_model_name}'
input_shape = str([1] + model.input.get_shape().as_list()[1:]).replace(" ", "")
data_type = 'FP32'
docker_output_model_path = f'{docker_mnt_vol}/IR_sdf/'
ov_model_name = 'vgg_sdf'
configs = f"--input_model {docker_frozen_model_path} \
            --input_shape {input_shape} \
            --data_type {data_type} \
            --output_dir {docker_output_model_path}  \
            --model_name {ov_model_name}"

model_optimizer_cmd = f"docker run -v {phys_mnt_vol}:{docker_mnt_vol} {os_version} /bin/bash executables/mo.sh {configs}"
model_optimizer_cmd # Checking the command to see if this works

'docker run -v /home/sdp/freyes/workspace/official_openseismic/updated_open_seismic/SFD-CNN-TL/models/tensorflow_conversion:/mnt_vol os:opensource_resmnt /bin/bash executables/mo.sh --input_model /mnt_vol/vgg_sfd.pb             --input_shape [1,45,45,1]             --data_type FP32             --output_dir /mnt_vol/IR_sdf/              --model_name vgg_sdf'

3. Run the model optimizer on Open Seismic!

In [10]:
! {model_optimizer_cmd}

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/mnt_vol/vgg_sfd.pb
	- Path for generated IR: 	/mnt_vol/IR_sdf/
	- IR output name: 	vgg_sdf
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,45,45,1]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output node names: 	None
	- Use configuration file used to generate the model with Object Det

Congratulations! You have successfully converted a Keras model to a Tensorflow frozen graph and optimized it to an OpenVINO IR!